# Построение финансовой модели в Python

где-то здесь в комментариях ссылка на оригинал: https://www.youtube.com/watch?v=ozq2W5iTmx4&list=PLpSi7aN6q3de2mpy5dyOjnG7QUguLt045&index=12&t=2224s&pp=iAQB

In [7]:
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib.pyplot as plt

import datetime as dt
from dateutil.relativedelta import relativedelta

import yfinance as yf

jtplot.style(figsize=(15, 9))

In [2]:
def extract_value(cell_range):
    result = []
    for cell in cell_range[0]:
        result.append(abs(cell.value))
    return result


def get_the_average(yearly_values):
    return np.array(yearly_values).mean()


def define_growth(values):
    change_rates = []
    for i in range(1, len(values)):
        change_rate = values[i]/values[i-1] - 1
        change_rates.append(change_rate)
    return change_rates



In [8]:
class Model:
    hist_revenues            = []
    hist_gross_profit        = []
    hist_net_interest        = []
    his_depreciation         = []
    hist_op_profit           = []
    hist_depr                = []
    historical_tax_rate      = []
    hist_NI                  = []
    hist_gross_profit_prct   = []
    hist_net_interest_prct   = []
    hist_depreciation_prct   = []
    hist_op_profit_prct      = []
    hist_depr_prct           = []
    historical_tax_rate_prct = []
    
    normalized= [
        hist_gross_profit_prct,
        hist_net_interest_prct,
        hist_op_profit_prct,
        hist_depr_prct,
    ]

    average_rates = {'gross_profit': 0, 'net_interest': 0, 'operating_profit': 0, 'depreciation': 0,}
    
    def __init__(self, company):
        self.ticker = company

    def define_revenue(self, cell_range):
        self.hist_revenues = extract_value(cell_range)

    def define_gross_profit(self, cell_range):
        self.hist_gross_profit = extract_value(cell_range)

    def define_operating_profit(self, cell_range):
        self.hist_op_profit = extract_value(cell_range)
    
    def calculate_beta(self):
        #get the dates
        today = dt.datetime.today()
        five_years_ago = today - relativedelta(years=5)
        
        #скачиваем данные и выбираем нужные колонки
        stock_price = yf.download(self.ticker, five_years_ago, today, auto_adjust=True)
        stock_price = stock_price['Close']
        index_price = yf.download('SPY', five_years_ago, today, auto_adjust=True)
        index_price = index_price['Close']
        
        #находим изменения в процентах и выбираем значения после первой строчки
        stock_price = stock_price.pct_change(1)[1:]
        index_price = index_price.pct_change(1)[1:]
        x = index_price
        y = stock_price
        
        #высчитываем бету
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        self.beta = slope
    
    def define_historical_depreciation(self, cell_range):
        self.hist_depr = extract_value(cell_range)

    def define_net_ineterst(self, cell_range_expense, cell_range_income):
        hist_II = np.array(extract_value(cell_range_income))
        hist_IE = np.array(extract_value(cell_range_expense))
        self.hist_net_interest = hist_II - hist_IE

    def define_hist_tax_rate_prct(self, range):
        self.historical_tax_rate = extract_value(range)

    def define_hist_NI(self, range):
        self.hist_NI = extract_value(range)

    def percentage_slice(self):
        self.all_marks = [
            self.hist_gross_profit,
            self.hist_net_interest,
            self.hist_op_profit,
            self.hist_depr,
        ]

        for data_set_index in range(0, len(self.all_marks)):
            data_set = self.all_marks[data_set_index]
            for i in range(0, len(data_set)):
                pct_value = data_set[i]/self.hist_revenues[i]
                self.normalized[data_set_index].append(pct_value)

    def define_avg_PCT(self):
        for set_index in range( 0, len(self.normalized)):
            keys = list(self.average_rates.keys())
            key = keys[set_index]
            values = self.normalized[set_index]
            self.average_rates[key] = get_the_average(values)

    def revenue_growth_rate(self):
        self.hist_growth_rates = define_growth(self.hist_revenues)
        self.average_growth_rate = get_the_average(self.hist_growth_rates)

    def model_future_revenue(self, adjustment):
        last_actual_revenue = self.hist_revenues[-1]
        self.future_revenues = [last_actual_revenue]
        for future_year in range(1,7):
            growth_rate = (1 + self.average_growth_rate + adjustment)
            future_current_revenue = self.future_revenues[future_year-1] * growth_rate
            self.future_revenues.append(future_current_revenue)

        self.future_revenues = self.future_revenues[1:]

    def model_EBIT(self):
        self.future_EBIT = []
        EBIT_rate = self.average_rates['operating_profit']
        for revenue in self.future_revenues:
            modelled_EBIT = revenue*EBIT_rate
            self.future_EBIT.append(modelled_EBIT)

In [9]:
nvda = Model('NVDA')

In [10]:
nvda.calculate_beta()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
nvda.beta

1.7616662822739744